In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    DDD_Train_Decile1 = pd.read_pickle("DDD_Train_Decile1.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile1.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile1, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 1')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

Epoch 5/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5740 - acc: 0.7222 - val_loss: 0.5574 - val_acc: 0.7196
Epoch 6/150
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5723 - acc: 0.7239 - val_loss: 0.5487 - val_acc: 0.7284
Epoch 7/150
2264/2264 [==============================] - 2s 994us/step - loss: 0.5639 - acc: 0.7261 - val_loss: 0.5461 - val_acc: 0.7302
Epoch 8/150
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5663 - acc: 0.7314 - val_loss: 0.5415 - val_acc: 0.7284
Epoch 9/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5594 - acc: 0.7323 - val_loss: 0.5402 - val_acc: 0.7337
Epoch 10/150
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5549 - acc: 0.7363 - val_loss: 0.5393 - val_acc: 0.7354
Epoch 11/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5491 - acc: 0.7367 - val_loss: 0.5347 - val_acc: 0.7407
Epoch 12/150
2264/2264 [===========================

2264/2264 [==============================] - 2s 1ms/step - loss: 0.5180 - acc: 0.7491 - val_loss: 0.5098 - val_acc: 0.7496
Epoch 64/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5176 - acc: 0.7473 - val_loss: 0.5098 - val_acc: 0.7496
Epoch 65/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5181 - acc: 0.7602 - val_loss: 0.5098 - val_acc: 0.7496
Epoch 66/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5216 - acc: 0.7487 - val_loss: 0.5098 - val_acc: 0.7496
Epoch 67/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5176 - acc: 0.7549 - val_loss: 0.5099 - val_acc: 0.7496
Epoch 68/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5205 - acc: 0.7535 - val_loss: 0.5099 - val_acc: 0.7496
Epoch 69/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5223 - acc: 0.7549 - val_loss: 0.5099 - val_acc: 0.7496
Epoch 70/150
2264/2264 [==============================] - 3s 

2264/2264 [==============================] - 2s 1ms/step - loss: 0.5209 - acc: 0.7473 - val_loss: 0.5097 - val_acc: 0.7531
Epoch 124/150
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5169 - acc: 0.7571 - val_loss: 0.5097 - val_acc: 0.7531
Epoch 125/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5176 - acc: 0.7478 - val_loss: 0.5096 - val_acc: 0.7513
Epoch 126/150
2264/2264 [==============================] - 3s 1ms/step - loss: 0.5166 - acc: 0.7491 - val_loss: 0.5096 - val_acc: 0.7513
Epoch 127/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5169 - acc: 0.7553 - val_loss: 0.5096 - val_acc: 0.7513
Epoch 128/150
2264/2264 [==============================] - 2s 1000us/step - loss: 0.5181 - acc: 0.7527 - val_loss: 0.5096 - val_acc: 0.7513
Epoch 129/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5207 - acc: 0.7527 - val_loss: 0.5096 - val_acc: 0.7513
Epoch 130/150
2264/2264 [===========================

2264/2264 [==============================] - 2s 1ms/step - loss: 0.5489 - acc: 0.7434 - val_loss: 0.5329 - val_acc: 0.7425
Epoch 33/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5469 - acc: 0.7438 - val_loss: 0.5316 - val_acc: 0.7407
Epoch 34/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5450 - acc: 0.7425 - val_loss: 0.5287 - val_acc: 0.7354
Epoch 35/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5382 - acc: 0.7473 - val_loss: 0.5325 - val_acc: 0.7390
Epoch 36/150
2264/2264 [==============================] - 2s 963us/step - loss: 0.5451 - acc: 0.7465 - val_loss: 0.5299 - val_acc: 0.7337
Epoch 37/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.5381 - acc: 0.7420 - val_loss: 0.5257 - val_acc: 0.7372
Epoch 38/150
2264/2264 [==============================] - 2s 976us/step - loss: 0.5434 - acc: 0.7451 - val_loss: 0.5286 - val_acc: 0.7337
Epoch 39/150
2264/2264 [==============================] -

...

2264/2264 [==============================] - 1s 525us/step - loss: 0.5521 - acc: 0.7394 - val_loss: 0.5433 - val_acc: 0.7319
Epoch 64/100
2264/2264 [==============================] - 1s 532us/step - loss: 0.5517 - acc: 0.7376 - val_loss: 0.5430 - val_acc: 0.7319
Epoch 65/100
2264/2264 [==============================] - 1s 517us/step - loss: 0.5549 - acc: 0.7376 - val_loss: 0.5423 - val_acc: 0.7337
Epoch 66/100
2264/2264 [==============================] - 1s 502us/step - loss: 0.5507 - acc: 0.7425 - val_loss: 0.5422 - val_acc: 0.7319
Epoch 67/100
2264/2264 [==============================] - 1s 504us/step - loss: 0.5529 - acc: 0.7443 - val_loss: 0.5419 - val_acc: 0.7319
Epoch 68/100
2264/2264 [==============================] - 1s 511us/step - loss: 0.5568 - acc: 0.7328 - val_loss: 0.5414 - val_acc: 0.7372
Epoch 69/100
2264/2264 [==============================] - 1s 515us/step - loss: 0.5503 - acc: 0.7420 - val_loss: 0.5403 - val_acc: 0.7390
Epoch 70/100
2264/2264 [=======================

2264/2264 [==============================] - 1s 522us/step - loss: 0.5316 - acc: 0.7451 - val_loss: 0.5197 - val_acc: 0.7460
Epoch 22/200
2264/2264 [==============================] - 1s 516us/step - loss: 0.5329 - acc: 0.7412 - val_loss: 0.5195 - val_acc: 0.7478
Epoch 23/200
2264/2264 [==============================] - 1s 520us/step - loss: 0.5340 - acc: 0.7513 - val_loss: 0.5158 - val_acc: 0.7601
Epoch 24/200
2264/2264 [==============================] - 1s 510us/step - loss: 0.5274 - acc: 0.7482 - val_loss: 0.5169 - val_acc: 0.7531
Epoch 25/200
2264/2264 [==============================] - 1s 531us/step - loss: 0.5272 - acc: 0.7482 - val_loss: 0.5149 - val_acc: 0.7601
Epoch 26/200
2264/2264 [==============================] - 1s 516us/step - loss: 0.5324 - acc: 0.7496 - val_loss: 0.5170 - val_acc: 0.7549
Epoch 27/200
2264/2264 [==============================] - 1s 528us/step - loss: 0.5311 - acc: 0.7420 - val_loss: 0.5164 - val_acc: 0.7549
Epoch 28/200
2264/2264 [=======================

Epoch 79/200
2264/2264 [==============================] - 1s 535us/step - loss: 0.5251 - acc: 0.7416 - val_loss: 0.5129 - val_acc: 0.7549
Epoch 80/200
2264/2264 [==============================] - 1s 517us/step - loss: 0.5233 - acc: 0.7531 - val_loss: 0.5130 - val_acc: 0.7531
Epoch 81/200
2264/2264 [==============================] - 1s 522us/step - loss: 0.5212 - acc: 0.7553 - val_loss: 0.5129 - val_acc: 0.7549
Epoch 82/200
2264/2264 [==============================] - 1s 514us/step - loss: 0.5234 - acc: 0.7496 - val_loss: 0.5130 - val_acc: 0.7531
Epoch 83/200
2264/2264 [==============================] - 1s 525us/step - loss: 0.5240 - acc: 0.7509 - val_loss: 0.5131 - val_acc: 0.7531
Epoch 84/200
2264/2264 [==============================] - 1s 520us/step - loss: 0.5245 - acc: 0.7482 - val_loss: 0.5131 - val_acc: 0.7531
Epoch 85/200
2264/2264 [==============================] - 1s 524us/step - loss: 0.5267 - acc: 0.7518 - val_loss: 0.5132 - val_acc: 0.7549
Epoch 86/200
2264/2264 [==========

2264/2264 [==============================] - 1s 541us/step - loss: 0.5303 - acc: 0.7496 - val_loss: 0.5128 - val_acc: 0.7531
Epoch 139/200
2264/2264 [==============================] - 1s 542us/step - loss: 0.5218 - acc: 0.7482 - val_loss: 0.5127 - val_acc: 0.7531
Epoch 140/200
2264/2264 [==============================] - 1s 517us/step - loss: 0.5268 - acc: 0.7456 - val_loss: 0.5128 - val_acc: 0.7531
Epoch 141/200
2264/2264 [==============================] - 1s 524us/step - loss: 0.5253 - acc: 0.7496 - val_loss: 0.5129 - val_acc: 0.7531
Epoch 142/200
2264/2264 [==============================] - 1s 530us/step - loss: 0.5228 - acc: 0.7487 - val_loss: 0.5128 - val_acc: 0.7531
Epoch 143/200
2264/2264 [==============================] - 1s 527us/step - loss: 0.5217 - acc: 0.7549 - val_loss: 0.5129 - val_acc: 0.7531
Epoch 144/200
2264/2264 [==============================] - 1s 528us/step - loss: 0.5255 - acc: 0.7482 - val_loss: 0.5130 - val_acc: 0.7549
Epoch 145/200
2264/2264 [================

2264/2264 [==============================] - 1s 523us/step - loss: 0.5215 - acc: 0.7562 - val_loss: 0.5126 - val_acc: 0.7531
Epoch 198/200
2264/2264 [==============================] - 1s 529us/step - loss: 0.5253 - acc: 0.7518 - val_loss: 0.5127 - val_acc: 0.7549
Epoch 199/200
2264/2264 [==============================] - 1s 513us/step - loss: 0.5218 - acc: 0.7460 - val_loss: 0.5126 - val_acc: 0.7531
Epoch 200/200
2264/2264 [==============================] - 1s 531us/step - loss: 0.5293 - acc: 0.7504 - val_loss: 0.5126 - val_acc: 0.7531
Test accuracy: 0.7530864191223494
Train on 2264 samples, validate on 567 samples
Epoch 1/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.6638 - acc: 0.5905 - val_loss: 0.6370 - val_acc: 0.6402
Epoch 2/50
2264/2264 [==============================] - 1s 510us/step - loss: 0.6236 - acc: 0.6413 - val_loss: 0.6062 - val_acc: 0.6402
Epoch 3/50
2264/2264 [==============================] - 1s 521us/step - loss: 0.6087 - acc: 0.6427 - val_lo

In [ ]:
0.86434